In [2]:
import re
import os, time, pickle
import torch
from omegaconf import OmegaConf
import hydra
import logging
from rfdiffusion.util import writepdb_multi, writepdb
from rfdiffusion.inference import utils as iu, symmetry
from hydra.core.hydra_config import HydraConfig
import numpy as np
import random
import glob

In [8]:
with hydra.initialize(version_base=None, config_path="./config/inference", job_name="base"):
    conf = hydra.compose(config_name="base")

contigs = '5/Y471-485/10-30/X95-101/10-30/X246-251/10-30/X265-271/1/X273-278/10-30/X313-322/5/0' #@param {type:"string"}
pdb = './examples/input_pdbs/1DBB_interface_30A.pdb' #@param {type:"string"}
num_designs = 1 #@param ["1", "2", "4", "8", "10", "16", "32"] {type:"raw"}
symmetry = "C4" #@param {type:"string"}

contigs_lst = []
for subcon in contigs.split("/"):
  if "-" in subcon and subcon[0].isdigit():
    length_inpaint = random.randint(int(subcon.split("-")[0]), int(subcon.split("-")[1]))
    contigs_lst.append(str(length_inpaint))
  else:
    contigs_lst.append(subcon)
new_contigs = "/".join(contigs_lst)


conf.inference.asy_motif = True
conf.contigmap.contigs = [new_contigs]
conf.inference.input_pdb = pdb
conf.inference.symmetry = symmetry

sampler = iu.sampler_selector(conf)

Reading models from /home/hychim/software/RFdiffusion_asy/rfdiffusion/inference/../../models
This is inf_conf.ckpt_path
/home/hychim/software/RFdiffusion_asy/rfdiffusion/inference/../../models/Base_ckpt.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][n_extra_block] = 4
USING MODEL CONFIG: self._conf[model][n_main_block] = 32
USING MODEL CONFIG: self._conf[model][n_ref_block] = 4
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USING MODEL CONFIG: self._conf[model][d_pair] = 128
USING MODEL CONFIG: self._conf[model][d_templ] = 64
USING MODEL CONFIG: self._conf[model][n_head_msa] = 8
USING MODEL CONFIG: self._conf[model][n_head_pair] = 4
USING MODEL CONFIG: self._conf[model][n_head_templ] = 4
USING MODEL CONFIG: self._conf[model][d_hidden] = 32
USING MODEL CONFIG: self._conf[model][d_hidden_templ] = 32
USING MODEL CONFIG: self._conf[model][p_drop] = 0.15
USING MODEL CONF

In [9]:
x_init, seq_init = sampler.sample_init()


With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


In [10]:
x_init

tensor([[[-6.7985, -1.3792, -3.9965],
         [-6.6705, -0.1656, -3.1986],
         [-7.2053,  1.0467, -3.9501],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[-3.1232,  1.7435, -5.4069],
         [-2.8869,  2.6726, -6.5054],
         [-2.8093,  1.9406, -7.8390],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 5.1294, -6.0108,  3.5204],
         [ 4.1139, -6.9922,  3.1580],
         [ 2.7342, -6.5602,  3.6380],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        ...,

        [[ 1.1310,  6.5706,  0.2067],
         [ 1.6731,  5.2584, -0.1248],
         [ 0.6615,  4.1562,  0.1618],
         ...,
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[